In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import numpy as np
import pandas as pd
from __future__ import print_function, division
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import time
import sys

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [8]:
#========================================================================
# First Month別にFitさせるにあたり、データセットの分布を各First Monthに揃える
#========================================================================

#========================================================================
# Args 
# fm
is_viz = False
is_max = True
limit_diff_num = 5
base_fam = '2017-09'
base_year = int(base_fam[:4])
base_month = int(base_fam[-2:])
max_fam = '2017-12' 
min_fam = '2011-11' 
# target_max = np.max(df.dropna().index.tolist())
# target_min = np.min(df.dropna().index.tolist())

result_id = []
#========================================================================

#========================================================================
# Data Load
base = utils.read_df_pkl('../input/base_first*')
base[target] = base[target].map(lambda x: np.round(x, 1))
#========================================================================
df_list = []

val_cnt = base[target].value_counts()
val_cnt.name = 'all'
df_list.append(val_cnt.copy())

base_1712 = base[base['first_active_month']==base_fam]
val_cnt = base_1712[target].value_counts()
# if is_max:
#     val_cnt_max = val_cnt.max()
#     val_cnt /= val_cnt_max
val_cnt.name = base_fam
df_list.append(val_cnt)
df = pd.concat(df_list, axis=1)
df.head()

100%|██████████| 3/3 [00:00<00:00, 30.78it/s]


,all,2017-09
-33.2,2207,35.0
-17.6,1,NaN
-17.5,1,NaN
-17.4,1,NaN
-17.2,1,NaN


,all,2017-12
-33.2,2207,NaN
-17.6,1,NaN
-17.5,1,NaN
-17.4,1,NaN
-17.2,1,NaN
-15.8,1,NaN
-15.6,1,NaN
-15.4,1,NaN
-15.3,1,NaN
-15.1,2,NaN


In [ ]:
def arange_ratio(df, multi, is_viz=False):
    df[base_fam] *= multi
    df['diff'] = df['all'] - df[base_fam]
    diff_len = len(df[df['diff']<0])
    if is_viz:
        display(df[df['diff']<0])
    if diff_len > limit_diff_num:
        return -1
    return 0
    
df = pd.concat(df_list, axis=1)
cnt_0_fam = df.loc[0.0, :][base_fam]
cnt_0_all = df.loc[0.0, :]['all']
multi = int(cnt_0_all / cnt_0_fam)+1

while True:
    tmp = df.copy().dropna()
    is_minus = arange_ratio(tmp, multi)
    if is_minus:
        multi -= 1
        continue
    break
    
print(f"multi: {multi}")
df[base_fam] *= multi
df_loy = df.dropna()
loy_list = list(df_loy.index)


#========================================================================
# Sampling
#========================================================================
for i in loy_list:
    loy = np.round(i, 1)
    df_id = base[base[target]==loy]
    if len(df_id)==0:
        continue
    sample = df.loc[loy, base_fam]
    if sample==sample:
        before = sample
    else:
        sample = before
    sample = np.int(sample)
    remain = sample
    sampling_id = []
    
    if is_viz:
        print('''
#========================================================================
# Sampling Start!!
''')
    
    for i in range(100):
        
        is_add = True
        if i==0:
            yyyymm = base_fam
            tmp_id = df_id[df_id['first_active_month']==yyyymm]
        else:
            year = base_year
            month = base_month + i
            
            if month>12:
                num_year = month//12
                year = year + num_year
                month = month - 12 * num_year
            elif month<1:
                num_year = month//12
                if num_year==0:
                    year = year - 1
                    month = month + 12
                else:
                    num_year *= -1
                    year = year - num_year
                    month = month + 12*num_year
            if month<10:
                month = f'0{month}'

            yyyymm = f"{year}-{month}"  

            if yyyymm<min_fam or yyyymm>max_fam:
                is_add = False
            else:
                tmp_id = df_id[df_id['first_active_month']==yyyymm]
                if i>0 and yyyymm==base_fam:
                    is_add = False

        #========================================================================
        # Sampling
        if is_add:
            if is_viz:
                print(f'future yyyymm: {yyyymm}')
            id_list = list(tmp_id[key].values)
            if len(id_list)<=remain:
                sampling_id += id_list
            else:
                sampling_id += list(np.random.choice(id_list, remain, replace=False))
                
            if is_viz:
                print(f"sampling_id: {len(sampling_id)} / {sample}")
        #========================================================================
                
        remain = sample - len(sampling_id)
        if remain <= 0:
            break
                
        is_add = True
        if i>0:
            year = base_year
            month = base_month - i
            
            if month>12:
                num_year = month//12
                year = year + num_year
                month = month - 12 * num_year
            elif month<1:
                num_year = month//12
                if num_year==0:
                    year = year - 1
                    month = month + 12
                else:
                    num_year *= -1
                    year = year - num_year
                    month = month + 12*num_year
            if month<10:
                month = f'0{month}'

            yyyymm = f"{year}-{month}"  
    
            if yyyymm<min_fam or yyyymm>max_fam:
                is_add = False
            else:
                tmp_id = df_id[df_id['first_active_month']==yyyymm]
            
            #========================================================================
            # Sampling
            if is_add:
                if is_viz:
                    print(f'past yyyymm: {yyyymm}')
                id_list = list(tmp_id[key].values)
                if len(id_list)<=remain:
                    sampling_id += id_list
                else:
                    sampling_id += list(np.random.choice(id_list, remain, replace=False))
    
                if is_viz:
                    print(f"sampling_id: {len(sampling_id)} / {sample}")
            #========================================================================
            
            remain = sample - len(sampling_id)
            if remain <= 0:
                break

    result_id += sampling_id
    if is_viz:
        print(f"loy:{loy} | {len(sampling_id)}/{sample} | All: {len(result_id)}")
        print('''
# Sampling Complete!!
#========================================================================
''')
print(f"All: {len(result_id)}")
display(base[base[key].isin(result_id)]['first_active_month'].value_counts().head(10))
display(base[base[key].isin(result_id)]['target'].value_counts().head(10))